In [23]:
import  pandas as pd
import os
os.chdir('/users/sgdbareh/volatile/ECHR_Importance/VectorDB/')
from run_experiments import select_embedding

/users/sgdbareh/volatile/miniconda3/envs/ECHR/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_pickle('/users/sgdbareh/volatile/ECHR_Importance/VectorDB/train.pkl')

In [3]:
outcome_case = pd.read_pickle('/users/sgdbareh/volatile/ECHR_Importance/Art_3_Data_Process/outcome_cases.pkl')

In [4]:
train = train[['Filename','Subject Matter','appno','extracted_apps']]

In [6]:
new_rows = []
for index, row in train.iterrows():
    for appno in row['extracted_apps']:
        new_row = {
            'Filename': row['Filename'],
            'Subject Matter': row['Subject Matter'],
            'Actual App No': row['appno'],
            'appno': appno
        }
        new_rows.append(new_row)

new_df = pd.DataFrame(new_rows)
#train = pd.concat([train, new_df], ignore_index=True)

In [7]:
new_df.rename(columns={'Subject Matter':'Comm_Case'}, inplace=True)

In [5]:
outcome_case['appno'] = outcome_case['appno'].str.split(';').str[0]

In [9]:
outcome_case.drop_duplicates(subset=['appno'], inplace=True,keep='last')

In [10]:
outcome_case = outcome_case[['File','appno']]

In [11]:
outcome_case.rename(columns={'File':'Filename'}, inplace=True)

In [13]:
summarise_data = pd.read_pickle('/users/sgdbareh/volatile/ECHR_Importance/Summarize_Cases/Results/outcome_summaries.pkl')

In [14]:
complete_outcome_data = pd.merge(outcome_case, summarise_data, on='Filename', how='inner')

In [18]:
training_data = pd.merge(new_df, complete_outcome_data, on='appno', how='left')

In [19]:
training_data.columns

Index(['Filename_x', 'Comm_Case', 'Actual App No', 'appno', 'Filename_y',
       '200 Word Summary', '500 Word Summary'],
      dtype='object')

In [20]:
training_data = training_data[['Filename_x','Comm_Case','appno','200 Word Summary']]

In [21]:
training_data.rename(columns={'Filename_x':'Filename','200 Word Summary':'Positive'}, inplace=True)

In [24]:
#FIND NEGATIVE CASES
results = pd.read_pickle('/users/sgdbareh/volatile/ECHR_Importance/VectorDB/cosine_openai_raw_chunk_1024_results.pkl')

In [25]:
for i,j in results.items():

    result_list = []

    for k in j:
        if k == '':
            continue    
        result_list.append(k.split(';')[0])  

    results[i] = result_list[15:]   

In [26]:
negative_candidates = pd.DataFrame.from_dict(results, orient='index')

In [27]:
neg_numbers = {}

for index, row in training_data.iterrows():
    # Access the values of each column in the row
    column1_value = row['Filename']
    neg_list = negative_candidates.loc[column1_value].values
    if column1_value not in neg_numbers.keys():
        neg_numbers[column1_value] = 49
    
    #print(neg_list)
    
    neg_candidate = neg_numbers[column1_value]

    while neg_candidate is None:
        neg_numbers[column1_value] -= 1
        neg_candidate = neg_list[neg_numbers[column1_value]]

    neg_numbers[column1_value] -= 1

    #print(neg_candidate)

    training_data.at[index, 'Negative'] = neg_list[neg_candidate]

In [28]:
complete_outcome_data.rename(columns={'appno':'Negative'}, inplace=True)

In [42]:
training_data_pos_neg = pd.merge(training_data, complete_outcome_data, on='Negative', how='inner')

In [43]:
training_data_pos_neg = training_data_pos_neg[['Filename_x','Comm_Case','Positive','200 Word Summary']]

In [44]:
training_data_pos_neg.rename(columns={'Filename_x':'Filename','200 Word Summary':'Negative'}, inplace=True)

In [45]:
#replace comm case facts with summaries

comm_summaries = pd.read_pickle('/users/sgdbareh/volatile/ECHR_Importance/Summarize_Cases/Results/comm_train_summaries.pkl')

In [46]:
training_data_pos_neg = pd.merge(training_data_pos_neg, comm_summaries, on='Filename', how='inner')

In [48]:
training_data_pos_neg.drop(columns=['Comm_Case','500 Word Summary'], inplace=True)

In [49]:
training_data_pos_neg.rename(columns={'200 Word Summary':'Comm_Case'}, inplace=True)

In [54]:
training_data_pos_neg.to_pickle('/users/sgdbareh/volatile/ECHR_Importance/BERT-rerank/BERT_training_data_df.pkl')

In [52]:
training_data_pos_neg

,Filename,Positive,Negative,Comm_Case
0,001-194607,The case involves three applicants who were de...,"The applicants, a mother and daughter from Chi...","The case involves four Russian women, Natalya ..."
1,001-194607,The case involves the death of Mr. Atanas Veli...,"The case involves the first applicant, a 17-ye...","The case involves four Russian women, Natalya ..."
2,001-194607,"The applicant, a police officer, was arrested ...",The case involves Romanian citizens of Roma or...,"The case involves four Russian women, Natalya ..."
3,001-194607,"The case involves three siblings, X, Y, and Z,...","The applicant, born in 1976 and residing in Si...","The case involves four Russian women, Natalya ..."
4,001-194607,"The case involves Mr. Halil İbrahim Okkalı, wh...","The applicant, born in 1982 and diagnosed with...","The case involves four Russian women, Natalya ..."
...,...,...,...,...
2311,001-145069,"Mr. Antonio Messina, an Italian national born ...","Mr. Julian Rosengren, a dual national of Roman...","Mr. Boris Mozer, a Moldovan national, was deta..."
2312,001-145069,The case concerns the Turkish military operati...,"Aleksandr Vyacheslavovich Medvedev, a Russian ...","Mr. Boris Mozer, a Moldovan national, was deta..."
2313,001-145069,The applicant was detained on remand in Cracow...,"The applicant, a school teacher with a third-d...","Mr. Boris Mozer, a Moldovan national, was deta..."
2314,001-176188,"The applicant, born in 1957, was the secretary...","Mr. Nugzar Tabagari, a Georgian national born ...",The case concerns an applicant who was remande...


In [53]:
from sentence_transformers import InputExample


In [ ]:
# Initialize an empty list for train_samples
train_samples = []

# Iterate over the DataFrame rows
for index, row in df.iterrows():
    # Create InputExample for positive label
    train_samples.append(InputExample(texts=[row['comm_case'], row['positive']], label=label2int["pos"]))
    # Create InputExample for negative label
    train_samples.append(InputExample(texts=[row['comm_case'], row['negative']], label=label2int["neg"]))


In [ ]:
# REPLACE COMM CASE WITH SUMMARY TOO!!!!

In [ ]:
#for loop

# for each case in train_subset
# find the text of a extracted appno and pair it as positive
# then find not very similar case and pair as negative do this for all extracted app nos

### use openai 1024 to find the least similar


## split this 80/20 cros validation to train 5 different models for train/validation


#eval on test set 

#label2int = {"neg": 0, "pos": 1}
#train_samples = [
#    InputExample(texts=["sentence1", "sentence2"], label=label2int["neg"]),
#    InputExample(texts=["Another", "pair"], label=label2int["pos"]),
#]